In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
# import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import trimesh
import torch
from evaluation.eval_meshes import MeshEvaluator
from utils.io import load_pointcloud
import yaml
from data import dataset

In [3]:
path="generation.yaml"
with open(path, 'r') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Shorthands
out_dir = cfg['training']['out_dir']
# generation_dir = os.path.join(out_dir, cfg['generation']['generation_dir'])
generation_dir = "out/pretrained"
#generation_dir = "out/pretrained_model_meshes"
out_file = os.path.join(generation_dir, 'eval_meshes_full.pkl')
out_file_class = os.path.join(generation_dir, 'eval_meshes.csv')

In [5]:
# Dataset
points_field = dataset.PointsField(
    cfg['data']['points_iou_file'], 
    unpackbits=cfg['data']['points_unpackbits'],
)
pointcloud_field = dataset.PointCloudField(
    cfg['data']['pointcloud_chamfer_file']
)
fields = {
    'points_iou': points_field,
    'pointcloud_chamfer': pointcloud_field,
    'idx': dataset.IndexField(),
}

print('Test split: ', cfg['data']['test_split'])

dataset_folder = cfg['data']['path']
dataset = dataset.Shapes3dDataset(
    dataset_folder, fields,
    cfg['data']['test_split'],
    categories=cfg['data']['classes'])

Test split:  test
Test split:  test


In [6]:
# Evaluator
evaluator = MeshEvaluator(n_points=100000)

# Loader
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, num_workers=0, shuffle=False)

In [7]:
torch.cuda.empty_cache()

In [8]:
# Evaluate all classes
eval_dicts = []
print('Evaluating meshes...')
for it, data in enumerate(tqdm(test_loader)):
    if data is None:
        print('Invalid data.')
        continue

    # Output folders
#     if not args.eval_input:
#         mesh_dir = os.path.join(generation_dir, 'meshes')
#         pointcloud_dir = os.path.join(generation_dir, 'pointcloud')
#     else:
    mesh_dir = os.path.join(generation_dir, 'meshes')
    pointcloud_dir = os.path.join(generation_dir, 'pointcloud')

    # Get index etc.
    idx = data['idx'].item()

    try:
        model_dict = dataset.get_model_dict(idx)
    except AttributeError:
        model_dict = {'model': str(idx), 'category': 'n/a'}
    
    modelname = model_dict['model']
    category_id = model_dict['category']

    try:
        category_name = dataset.metadata[category_id].get('name', 'n/a')
    except AttributeError:
        category_name = 'n/a'

    if category_id != 'n/a':
        mesh_dir = os.path.join(mesh_dir, category_id)
        pointcloud_dir = os.path.join(pointcloud_dir, category_id)

    # Evaluate
    pointcloud_tgt = data['pointcloud_chamfer'].squeeze(0).numpy()
    normals_tgt = data['pointcloud_chamfer.normals'].squeeze(0).numpy()
    points_tgt = data['points_iou'].squeeze(0).numpy()
    occ_tgt = data['points_iou.occ'].squeeze(0).numpy()

    # Evaluating mesh and pointcloud
    # Start row and put basic informatin inside
    eval_dict = {
        'idx': idx,
        'class id': category_id,
        'class name': category_name,
        'modelname': modelname,
    }
    eval_dicts.append(eval_dict)

    # Evaluate mesh
    if cfg['test']['eval_mesh']:
        mesh_file = os.path.join(mesh_dir, '%s.off' % modelname)

        if os.path.exists(mesh_file):
            mesh = trimesh.load(mesh_file, process=False)
            eval_dict_mesh = evaluator.eval_mesh(
                mesh, pointcloud_tgt, normals_tgt, points_tgt, occ_tgt)
            for k, v in eval_dict_mesh.items():
                eval_dict[k + ' (mesh)'] = v
        else:
            print('Warning: mesh does not exist: %s' % mesh_file)

    # Evaluate point cloud
    if cfg['test']['eval_pointcloud']:
        pointcloud_file = os.path.join(
            pointcloud_dir, '%s.ply' % modelname)

        if os.path.exists(pointcloud_file):
            pointcloud = load_pointcloud(pointcloud_file)
            eval_dict_pcl = evaluator.eval_pointcloud(
                pointcloud, pointcloud_tgt)
            for k, v in eval_dict_pcl.items():
                eval_dict[k + ' (pcl)'] = v
        else:
            print('Warning: pointcloud does not exist: %s'
                    % pointcloud_file)

Evaluating meshes...
Evaluating meshes...


 27%|█████████████████████▉                                                           | 2376/8751 [16:31<1:18:52,  1.35it/s]

 57%|███████████████████████████████████████████████▎                                   | 4992/8751 [46:55<50:17,  1.25it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 8751/8751 [1:24:52<00:00,  1.72it/s]


In [9]:
# Create pandas dataframe and save
eval_df = pd.DataFrame(eval_dicts)
eval_df.set_index(['idx'], inplace=True)
eval_df.to_pickle(out_file)

# Create CSV file  with main statistics
eval_df_class = eval_df.drop('modelname', axis=1).groupby(by=['class name']).mean()
eval_df_class.to_csv(out_file_class)

# Print results
eval_df_class.loc['mean'] = eval_df_class.mean()
print(eval_df_class)


                                                   class id  \
class name                                                    
airplane,aeroplane,plane                                inf   
bench                                                   inf   
cabinet                                                 inf   
car,auto,automobile,machine,motorcar                    inf   
chair                                                   inf   
display,video display                                   inf   
lamp                                                    inf   
loudspeaker,speaker,speaker unit,loudspeaker sy...      inf   
rifle                                                   inf   
sofa,couch,lounge                                       inf   
table                                                   inf   
telephone,phone,telephone set                           inf   
vessel,watercraft                                       inf   
mean                                                   

                                                   class id  \
class name                                                    
airplane,aeroplane,plane                                inf   
bench                                                   inf   
cabinet                                                 inf   
car,auto,automobile,machine,motorcar                    inf   
chair                                                   inf   
display,video display                                   inf   
lamp                                                    inf   
loudspeaker,speaker,speaker unit,loudspeaker sy...      inf   
rifle                                                   inf   
sofa,couch,lounge                                       inf   
table                                                   inf   
telephone,phone,telephone set                           inf   
vessel,watercraft                                       inf   
mean                                                   

In [10]:
# Create pandas dataframe and save
eval_df = pd.DataFrame(eval_dicts)
eval_df.set_index(['idx'], inplace=True)
eval_df

,class id,class name,modelname,completeness (mesh),accuracy (mesh),normals completeness (mesh),normals accuracy (mesh),normals (mesh),completeness2 (mesh),accuracy2 (mesh),chamfer-L2 (mesh),chamfer-L1 (mesh),iou (mesh),chamfer (mesh)
idx,,,,,,,,,,,,,,
0,02691156,"airplane,aeroplane,plane",d18592d9615b01bbbc0909d98a1ff2b4,0.056841,0.020662,0.626500,0.765092,0.695796,0.007578,0.000627,0.004103,0.038751,0.304348,NaN
1,02691156,"airplane,aeroplane,plane",d18f2aeae4146464bd46d022fd7d80aa,0.038793,0.016124,0.674373,0.794593,0.734483,0.004330,0.000350,0.002340,0.027459,0.370813,NaN
2,02691156,"airplane,aeroplane,plane",d199612c22fe9313f4fb6842b3610149,0.041992,0.018134,0.650054,0.712112,0.681083,0.004905,0.000488,0.002697,0.030063,0.271663,NaN
3,02691156,"airplane,aeroplane,plane",d1a887a47991d1b3bc0909d98a1ff2b4,0.045096,0.017325,0.629339,0.782354,0.705846,0.005317,0.000431,0.002874,0.031211,0.365782,NaN
4,02691156,"airplane,aeroplane,plane",d1a8e79eebf4a0b1579c3d4943e463ef,0.055582,0.020457,0.590429,0.794569,0.692499,0.007230,0.000579,0.003905,0.038020,0.333981,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,03001627,chair,u6028f63e-4111-4412-9098-fe5f4f0c7c83,0.229026,0.142582,0.423708,0.312294,0.368001,0.077058,0.032681,0.054869,0.185804,0.049555,NaN
8747,03001627,chair,ub5d972a1-de16-4d0a-aa40-85cd3a69aa8a,0.034557,0.031462,0.696457,0.705142,0.700800,0.002846,0.002005,0.002426,0.033009,0.234830,NaN
8748,03001627,chair,uca24feec-f0c0-454c-baaf-561530686f40,0.023801,0.024430,0.679383,0.713110,0.696247,0.000949,0.000973,0.000961,0.024116,0.316350,NaN


,class id,class name,modelname,completeness (mesh),accuracy (mesh),normals completeness (mesh),normals accuracy (mesh),normals (mesh),completeness2 (mesh),accuracy2 (mesh),chamfer-L2 (mesh),chamfer-L1 (mesh),iou (mesh),chamfer (mesh)
idx,,,,,,,,,,,,,,
0,02691156,"airplane,aeroplane,plane",d18592d9615b01bbbc0909d98a1ff2b4,0.056841,0.020662,0.626500,0.765092,0.695796,0.007578,0.000627,0.004103,0.038751,0.304348,NaN
1,02691156,"airplane,aeroplane,plane",d18f2aeae4146464bd46d022fd7d80aa,0.038793,0.016124,0.674373,0.794593,0.734483,0.004330,0.000350,0.002340,0.027459,0.370813,NaN
2,02691156,"airplane,aeroplane,plane",d199612c22fe9313f4fb6842b3610149,0.041992,0.018134,0.650054,0.712112,0.681083,0.004905,0.000488,0.002697,0.030063,0.271663,NaN
3,02691156,"airplane,aeroplane,plane",d1a887a47991d1b3bc0909d98a1ff2b4,0.045096,0.017325,0.629339,0.782354,0.705846,0.005317,0.000431,0.002874,0.031211,0.365782,NaN
4,02691156,"airplane,aeroplane,plane",d1a8e79eebf4a0b1579c3d4943e463ef,0.055582,0.020457,0.590429,0.794569,0.692499,0.007230,0.000579,0.003905,0.038020,0.333981,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,03001627,chair,u6028f63e-4111-4412-9098-fe5f4f0c7c83,0.229026,0.142582,0.423708,0.312294,0.368001,0.077058,0.032681,0.054869,0.185804,0.049555,NaN
8747,03001627,chair,ub5d972a1-de16-4d0a-aa40-85cd3a69aa8a,0.034557,0.031462,0.696457,0.705142,0.700800,0.002846,0.002005,0.002426,0.033009,0.234830,NaN
8748,03001627,chair,uca24feec-f0c0-454c-baaf-561530686f40,0.023801,0.024430,0.679383,0.713110,0.696247,0.000949,0.000973,0.000961,0.024116,0.316350,NaN
